In [ ]:
import random
%pip install --upgrade firebase-admin
# %pip install  firebase-admin
# %pip install requests==1.1.0
# %pip install python-firebase

Requirement already up-to-date: firebase-admin in /usr/local/lib/python3.7/dist-packages (5.0.0)


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/My\ Drive/'Colab Notebooks'/'SentiGAN-persuasion'

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/SentiGAN-persuasion


In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore, auth


cred = credentials.Certificate("Real_dataset/message-generation-firebase-adminsdk-o7l95-d2655c10e4.json")
firebase_admin.initialize_app(cred)
db = firestore.client()


ValueError: ignored


0.5 - 0.5
1.   Commitment ->  274
2.   Liking -> 334
3.   Consistency -> 374

0.6 - 0.6
1.   Commitment ->  253
2.   Liking -> 299
3.   Consistency -> 340

0.7 - 0.7
1.   Commitment ->  169
2.   Liking -> 206
3.   Consistency -> 229

In [ ]:
model = "Consistency"
persuation_colect = db.collection(model)
novelty=persuation_colect.where(u'novelty', u'>', 0.6).order_by(u'novelty').get()
diversity=persuation_colect.where(u'diversity', u'>', 0.6).order_by(u'diversity').get()
# docs = persuation_colect.stream()
docs = list(set(novelty) & set(diversity))
print("0.6 0.5")
print(len(docs))

0.6 0.5
340


In [ ]:
models = ["Commitment","Liking","Consistency"]

persuation_colect = db.collection(models[2])
novelty=persuation_colect.where(u'novelty', u'>', 0.73).order_by(u'novelty').get()
diversity=persuation_colect.where(u'diversity', u'>', 0.73).order_by(u'diversity').get()
docs = persuation_colect.get()
docs = list(set(novelty))
docs = list(set(novelty).intersection(set(diversity)))

one_word_list=[]
newdocs= []
for doc in docs:
  #  print(doc.to_dict()['message'])
  if len(doc.to_dict()['message'].split()) <= 8:
    
    one_word_list.append(doc.to_dict())
  else:
    # print(doc.to_dict()['message'])
    newdocs.append(doc.to_dict())

print(list(newdocs))
print(len(newdocs))

[{'diversity': 0.75, 'order': 33, 'message': 'aw i just you know ive been talking to my pilot and you will get an arbitrator', 'novelty': 0.7391304347826086, 'id': 'XpsEuJevXCIUjjil6928'}, {'diversity': 0.8181818181818181, 'order': 256, 'novelty': 0.7727272727272727, 'message': 'you now im tellin you one thing and this is not intelligent this is for forgiveness and', 'id': 'LRUsA8bqCdmB63RlyHva'}, {'diversity': 0.7916666666666666, 'order': 306, 'novelty': 0.7619047619047619, 'id': 'VrkShNHSkYX2O35ZmXk4', 'message': 'the white horse the first thing that i try to teach people i mean to get me'}, {'order': 319, 'novelty': 0.8421052631578947, 'message': 'i guess theyre why dont <ht02> is sittin over there playing', 'group_code': 'Con010', 'diversity': 0.8571428571428572, 'id': 'R88KQJFMV72Ea4KrEy77'}, {'diversity': 0.75, 'order': 217, 'id': '87iYGwKV3lNHNV11PBQv', 'novelty': 0.84, 'message': 'imagine vain things for the leaders of the earth and the rulers take counsel together against the'

In [ ]:
def delete_groups():
  models = ["Commitment","Liking","Consistency"]
  for model in models :
    docs = db.collection(model).stream()
    for doc in docs:
      #  print(f'{doc.id} => {doc.to_dict()}')
      doc.reference.update({
            u'group_code': firestore.DELETE_FIELD,
      })
        
#delete_groups()

In [ ]:
def filter_collection(model):
  persuation_colect = db.collection(model)
  if(model=='Commitment'):
    novelty = persuation_colect.where(u'novelty', u'>', 0.6).order_by(u'novelty').get()
    diversity = persuation_colect.where(u'diversity', u'>', 0.6).order_by(u'diversity').get()
  else:
    novelty = persuation_colect.where(u'novelty', u'>', 0.7).order_by(u'novelty').get()
    diversity = persuation_colect.where(u'diversity', u'>', 0.7).order_by(u'diversity').get()
  docs = persuation_colect.get()
  docs = list(set(novelty))
  docs = list(set(novelty).intersection(set(diversity)))

  one_word_list=[]
  newdocs= []
  for doc in docs:
    #  print(doc.to_dict()['message'])
    if len(doc.to_dict()['message'].split()) <= 8:
      one_word_list.append(doc.to_dict())
    else:
      # print(doc.to_dict()['message'])
      newdocs.append(doc)
  return newdocs

In [ ]:
def save_on_database(model,id,name):
  persuation_colect = db.collection(model)
  persuation_colect.document(id).update({
      u'group_code':name,
  }) 

In [ ]:
def set_groups(model,docs,codes_dic,groups=20):
  random.Random(10).shuffle(docs)
  group_size=5
  count=0
  ngroup=1
  name=model[0:3]+str(ngroup).zfill(3);
  codes=[]
  for doc in docs:
      if(count%group_size==0 and ngroup <= groups):
        name=model[0:3]+str(ngroup).zfill(3);
        ngroup+=1
        codes.append(name)
      if(count<groups*group_size):
        save_on_database(model, doc.id,name)
        dic=doc.to_dict()
        # print(f'{name, dic["novelty"],dic["diversity"]} => {dic["message"]}')
      count+=1
  codes_dic[model]=codes

In [ ]:
models = ["Commitment","Liking","Consistency"]
codes_dic={}
delete_groups()
for model in models:
   docs=filter_collection(model)
   set_groups(model,docs,codes_dic)
   print(len(docs))
print(codes_dic)

126
189
203
{'Commitment': ['Com001', 'Com002', 'Com003', 'Com004', 'Com005', 'Com006', 'Com007', 'Com008', 'Com009', 'Com010', 'Com011', 'Com012', 'Com013', 'Com014', 'Com015', 'Com016', 'Com017', 'Com018', 'Com019', 'Com020'], 'Liking': ['Lik001', 'Lik002', 'Lik003', 'Lik004', 'Lik005', 'Lik006', 'Lik007', 'Lik008', 'Lik009', 'Lik010', 'Lik011', 'Lik012', 'Lik013', 'Lik014', 'Lik015', 'Lik016', 'Lik017', 'Lik018', 'Lik019', 'Lik020'], 'Consistency': ['Con001', 'Con002', 'Con003', 'Con004', 'Con005', 'Con006', 'Con007', 'Con008', 'Con009', 'Con010', 'Con011', 'Con012', 'Con013', 'Con014', 'Con015', 'Con016', 'Con017', 'Con018', 'Con019', 'Con020']}


In [ ]:
codes_dic={'Commitment': ['Com001', 'Com002', 'Com003', 'Com004', 'Com005', 'Com006', 'Com007', 'Com008', 'Com009', 'Com010', 'Com011', 'Com012', 'Com013', 'Com014', 'Com015', 'Com016', 'Com017', 'Com018', 'Com019', 'Com020'], 'Liking': ['Lik001', 'Lik002', 'Lik003', 'Lik004', 'Lik005', 'Lik006', 'Lik007', 'Lik008', 'Lik009', 'Lik010', 'Lik011', 'Lik012', 'Lik013', 'Lik014', 'Lik015', 'Lik016', 'Lik017', 'Lik018', 'Lik019', 'Lik020'], 'Consistency': ['Con001', 'Con002', 'Con003', 'Con004', 'Con005', 'Con006', 'Con007', 'Con008', 'Con009', 'Con010', 'Con011', 'Con012', 'Con013', 'Con014', 'Con015', 'Con016', 'Con017', 'Con018', 'Con019', 'Con020']}
'abejar@unsa.edu.pe','alajoa@unsa.edu.pe',
'dpumaraime@unsa.edu.pe','eespinozape@unsa.edu.pe',
'ehuancap@unsa.edu.pe',
'ehuayllasco@unsa.edu.pe',
'epalli@unsa.edu.pe',
'equinonez@unsa.edu.pe',
'etorresq@unsa.edu.pe',
'evargashe@unsa.edu.pe',
'fflorezag@unsa.edu.pe',
'hyanquiv@unsa.edu.pe',
'imoscosoa@unsa.edu.pe',
'jchavezgue@unsa.edu.pe',
'jcuevaf@unsa.edu.pe',
'jhuaihuah@unsa.edu.pe',
'jperca@unsa.edu.pe',
'jtaccag@unsa.edu.pe',
'jvaldiviame@unsa.edu.pe',
'lmoroco@unsa.edu.pe',
'lsantet@unsa.edu.pe',
'lvillanuevag@unsa.edu.pe',
'lvillanuevaf@unsa.edu.pe',
'mherreraco@unsa.edu.pe',
'mhilaresa@unsa.edu.pe',
'mvilchezmo@unsa.edu.pe',
'scutipaay@unsa.edu.pe',
'sromeroc@unsa.edu.pe',
'svilcalim@unsa.edu.pe',

participantes=[
               'elipau@unsa.edu.pe','edson.lipa@gmail.com',
               'enriquewolf27@gmail.com','lmurillo@unsa.edu.pe',
               'amendozavil@unsa.edu.pe','pfloresfernandez14@gmail.com']

In [ ]:

codes_dic={'Commitment': ['Com001', 'Com002', 'Com003', 'Com004', 'Com005', 'Com006', 'Com007', 'Com008', 'Com009', 'Com010', 'Com011', 'Com012', 'Com013', 'Com014', 'Com015', 'Com016', 'Com017', 'Com018', 'Com019', 'Com020'], 'Liking': ['Lik001', 'Lik002', 'Lik003', 'Lik004', 'Lik005', 'Lik006', 'Lik007', 'Lik008', 'Lik009', 'Lik010', 'Lik011', 'Lik012', 'Lik013', 'Lik014', 'Lik015', 'Lik016', 'Lik017', 'Lik018', 'Lik019', 'Lik020'], 'Consistency': ['Con001', 'Con002', 'Con003', 'Con004', 'Con005', 'Con006', 'Con007', 'Con008', 'Con009', 'Con010', 'Con011', 'Con012', 'Con013', 'Con014', 'Con015', 'Con016', 'Con017', 'Con018', 'Con019', 'Con020']}

participantes=[
               'abejar@unsa.edu.pe','alajoa@unsa.edu.pe',
              'dpumaraime@unsa.edu.pe','eespinozape@unsa.edu.pe',
              'ehuancap@unsa.edu.pe','ehuayllasco@unsa.edu.pe',
              'epalli@unsa.edu.pe','equinonez@unsa.edu.pe',
              'etorresq@unsa.edu.pe','evargashe@unsa.edu.pe',
              'fflorezag@unsa.edu.pe','hyanquiv@unsa.edu.pe',
              'imoscosoa@unsa.edu.pe','jchavezgue@unsa.edu.pe',
              'jcuevaf@unsa.edu.pe','jhuaihuah@unsa.edu.pe',
              'jperca@unsa.edu.pe','jtaccag@unsa.edu.pe',
              'jvaldiviame@unsa.edu.pe','lmoroco@unsa.edu.pe',
              'lsantet@unsa.edu.pe','lvillanuevag@unsa.edu.pe',
              'lvillanuevaf@unsa.edu.pe','mherreraco@unsa.edu.pe',
              'mhilaresa@unsa.edu.pe','mvilchezmo@unsa.edu.pe',
              'scutipaay@unsa.edu.pe','sromeroc@unsa.edu.pe',
              'svilcalim@unsa.edu.pe','fran.slom@gmail.com',
               'ncfyllen7@gmail.com','elipau@unsa.edu.pe'
              ]
collection = db.collection(u'Participant')
docs=collection.stream()
for parti in docs:
  #print(f'{parti.id} => {parti.to_dict()}')
  collection.document(parti.id).delete()
code_index=0
for i in range(len(participantes)):
  if(i%2==0):
    collection = db.collection(u'Participant')
    new_parti=collection.document()
    array_codes=[codes_dic["Commitment"][code_index],codes_dic["Liking"][code_index],codes_dic["Consistency"][code_index]]
    random.shuffle(array_codes)
    new_parti.set({
        u'id':new_parti.id,
        u'email':participantes[i],
        u'done':False,
        u'code_1':array_codes[0],
        u'code_2':array_codes[1],
        u'code_3':array_codes[2]
    })
    new_parti=collection.document()
    random.shuffle(array_codes)
    new_parti.set({
        u'id':new_parti.id,
        u'email':participantes[i+1],
        u'done':False,
        u'code_1':array_codes[0],
        u'code_2':array_codes[1],
        u'code_3':array_codes[2]
    })
    code_index+=1
    
    print(participantes[i])
    print(participantes[i+1])

abejar@unsa.edu.pe
alajoa@unsa.edu.pe
dpumaraime@unsa.edu.pe
eespinozape@unsa.edu.pe
ehuancap@unsa.edu.pe
ehuayllasco@unsa.edu.pe
epalli@unsa.edu.pe
equinonez@unsa.edu.pe
etorresq@unsa.edu.pe
evargashe@unsa.edu.pe
fflorezag@unsa.edu.pe
hyanquiv@unsa.edu.pe
imoscosoa@unsa.edu.pe
jchavezgue@unsa.edu.pe
jcuevaf@unsa.edu.pe
jhuaihuah@unsa.edu.pe
jperca@unsa.edu.pe
jtaccag@unsa.edu.pe
jvaldiviame@unsa.edu.pe
lmoroco@unsa.edu.pe
lsantet@unsa.edu.pe
lvillanuevag@unsa.edu.pe
lvillanuevaf@unsa.edu.pe
mherreraco@unsa.edu.pe
mhilaresa@unsa.edu.pe
mvilchezmo@unsa.edu.pe
scutipaay@unsa.edu.pe
sromeroc@unsa.edu.pe
svilcalim@unsa.edu.pe
fran.slom@gmail.com
ncfyllen7@gmail.com
elipau@unsa.edu.pe


In [ ]:
codes_dic={'Commitment': ['Com001', 'Com002', 'Com003', 'Com004', 'Com005', 'Com006', 'Com007', 'Com008', 'Com009', 'Com010', 'Com011', 'Com012', 'Com013', 'Com014', 'Com015'], 'Liking': ['Lik001', 'Lik002', 'Lik003', 'Lik004', 'Lik005', 'Lik006', 'Lik007', 'Lik008', 'Lik009', 'Lik010', 'Lik011', 'Lik012', 'Lik013', 'Lik014', 'Lik015'], 'Consistency': ['Con001', 'Con002', 'Con003', 'Con004', 'Con005', 'Con006', 'Con007', 'Con008', 'Con009', 'Con010', 'Con011', 'Con012', 'Con013', 'Con014', 'Con015']}
participantes=[
               'elipau@unsa.edu.pe','edson.lipa@gmail.com',
               'enriquewolf27@gmail.com','lmurillo@unsa.edu.pe',
               'amendozavil@unsa.edu.pe','pfloresfernandez14@gmail.com']
def delete_ratings():
  for key,codes in codes_dic.items():
    ref = db.collection(key)
    print(key)
    for code in codes:
      docs=ref.where(u'group_code',u'==',code).stream()
      for doc in docs:
        #  print(f'{doc.id} => {doc.to_dict()}')
        doc.reference.update({
            u'ratings': firestore.DELETE_FIELD,
            u'rating_1': firestore.DELETE_FIELD,
            u'rating_2': firestore.DELETE_FIELD,
        })

  collection = db.collection(u'Participant')
  docs=collection.stream()
  for parti in docs:
    #print(f'{parti.id} => {parti.to_dict()}')
    parti.reference.update({
      u'done': False,
    })
delete_ratings()

Commitment
Liking
Consistency


Piloto con el grupo kusisqa

In [ ]:
models = ["Commitment","Liking","Consistency"]
codes_dic={}
# delete_groups()
for model in models:
   docs=filter_collection(model)
   set_groups(model,docs,codes_dic)
   print(len(docs))
print(codes_dic)

126
189
203
{'Commitment': ['Com001', 'Com002', 'Com003', 'Com004', 'Com005', 'Com006', 'Com007', 'Com008', 'Com009', 'Com010', 'Com011', 'Com012', 'Com013', 'Com014', 'Com015', 'Com016', 'Com017', 'Com018', 'Com019', 'Com020'], 'Liking': ['Lik001', 'Lik002', 'Lik003', 'Lik004', 'Lik005', 'Lik006', 'Lik007', 'Lik008', 'Lik009', 'Lik010', 'Lik011', 'Lik012', 'Lik013', 'Lik014', 'Lik015', 'Lik016', 'Lik017', 'Lik018', 'Lik019', 'Lik020'], 'Consistency': ['Con001', 'Con002', 'Con003', 'Con004', 'Con005', 'Con006', 'Con007', 'Con008', 'Con009', 'Con010', 'Con011', 'Con012', 'Con013', 'Con014', 'Con015', 'Con016', 'Con017', 'Con018', 'Con019', 'Con020']}


In [ ]:

participantes=['acunopa@unsa.edu.pe',
'amayhuaq@unsa.edu.pe',
'esarmientoca@unsa.edu.pe',
'pcalcinacc@unsa.edu.pe',
'abolanosc@unsa.edu.pe',
'adelgadom@unsa.edu.pe',
'ajbenavides@unsa.edu.pe',
'alvaroecp@gmail.com',
'amamaniali@unsa.edu.pe',
'amendozavil@unsa.edu.pe',
'wramos@unsa.edu.pe',
'fcruz@unsa.edu.pe',
'fsunilo@unsa.edu.pe',
'lbernal@unsa.edu.pe',
'mquispecr@unsa.edu.pe',
'ymamanimamani@unsa.edu.pe',
'edson.lipa@gmail.com',
'elipau@unsa.edu.pe',
]

codes_dic={'Commitment': ['Com001', 'Com002', 'Com003', 'Com004', 'Com005', 'Com006', 'Com007', 'Com008', 'Com009', 'Com010', 'Com011', 'Com012', 'Com013', 'Com014', 'Com015', 'Com016', 'Com017', 'Com018', 'Com019', 'Com020'], 'Liking': ['Lik001', 'Lik002', 'Lik003', 'Lik004', 'Lik005', 'Lik006', 'Lik007', 'Lik008', 'Lik009', 'Lik010', 'Lik011', 'Lik012', 'Lik013', 'Lik014', 'Lik015', 'Lik016', 'Lik017', 'Lik018', 'Lik019', 'Lik020'], 'Consistency': ['Con001', 'Con002', 'Con003', 'Con004', 'Con005', 'Con006', 'Con007', 'Con008', 'Con009', 'Con010', 'Con011', 'Con012', 'Con013', 'Con014', 'Con015', 'Con016', 'Con017', 'Con018', 'Con019', 'Con020']}

collection = db.collection(u'Participant')
docs=collection.stream()
for parti in docs:
  #print(f'{parti.id} => {parti.to_dict()}')
  collection.document(parti.id).delete()
code_index=0
for i in range(len(participantes)):
  if(i%2==0):
    collection = db.collection(u'Participant')
    new_parti=collection.document()
    array_codes=[codes_dic["Commitment"][code_index],codes_dic["Liking"][code_index],codes_dic["Consistency"][code_index]]
    random.shuffle(array_codes)
    new_parti.set({
        u'id':new_parti.id,
        u'email':participantes[i],
        u'done':False,
        u'code_1':array_codes[0],
        u'code_2':array_codes[1],
        u'code_3':array_codes[2]
    })
    new_parti=collection.document()
    random.shuffle(array_codes)
    new_parti.set({
        u'id':new_parti.id,
        u'email':participantes[i+1],
        u'done':False,
        u'code_1':array_codes[0],
        u'code_2':array_codes[1],
        u'code_3':array_codes[2]
    })
    code_index+=1
    
    print(participantes[i])
    print(participantes[i+1])

acunopa@unsa.edu.pe
amayhuaq@unsa.edu.pe
esarmientoca@unsa.edu.pe
pcalcinacc@unsa.edu.pe
abolanosc@unsa.edu.pe
adelgadom@unsa.edu.pe
ajbenavides@unsa.edu.pe
alvaroecp@gmail.com
amamaniali@unsa.edu.pe
amendozavil@unsa.edu.pe
wramos@unsa.edu.pe
fcruz@unsa.edu.pe
fsunilo@unsa.edu.pe
lbernal@unsa.edu.pe
mquispecr@unsa.edu.pe
ymamanimamani@unsa.edu.pe
edson.lipa@gmail.com
elipau@unsa.edu.pe
